In [14]:
import mappy as mp

file_path = r'/Users/disha/Desktop/Undergrad project/Data /Heliconius charithonia.fna'
file_path2 = r'/Users/disha/Desktop/Undergrad project/Data /Heliconius sara.fna'
a = mp.Aligner(file_path)  # load or build index
if not a: 
    raise Exception("ERROR: failed to load/build index")

results_dictionary={}
count=1
for name, seq, qual in mp.fastx_read(file_path2): # read a fasta/q sequence
        for hit in a.map(seq): # traverse alignments
                results_dictionary[count]=[hit.ctg,hit.r_st, hit.r_en, hit.cigar_str]
                print(f"done for {count}")
                count+=1
           #     print("{}\t{}\t{}\t{}".format(hit.ctg, hit.r_st, hit.r_en, hit.cigar_str))

done for 1
done for 2
done for 3
done for 4
done for 5
done for 6
done for 7
done for 8
done for 9
done for 10
done for 11
done for 12
done for 13
done for 14
done for 15
done for 16
done for 17
done for 18
done for 19
done for 20
done for 21
done for 22
done for 23
done for 24
done for 25
done for 26
done for 27
done for 28
done for 29
done for 30
done for 31
done for 32
done for 33
done for 34
done for 35
done for 36
done for 37
done for 38
done for 39
done for 40
done for 41
done for 42
done for 43
done for 44
done for 45
done for 46
done for 47
done for 48
done for 49
done for 50
done for 51
done for 52
done for 53
done for 54
done for 55
done for 56
done for 57
done for 58
done for 59
done for 60
done for 61
done for 62
done for 63
done for 64
done for 65
done for 66
done for 67
done for 68
done for 69
done for 70
done for 71
done for 72
done for 73
done for 74
done for 75
done for 76
done for 77
done for 78
done for 79
done for 80
done for 81
done for 82
done for 83
done for 84
d

KeyboardInterrupt: 

In [15]:
results_dictionary.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

In [5]:
import re

# Helper function to parse CIGAR and count matches/mismatches
def parse_cigar(cigar):
    matches = 0
    mismatches = 0
    indels = 0
    parsed_cigar = re.findall(r'(\d+)([MIDNSHP=X])', cigar)

    for length, op in parsed_cigar:
        length = int(length)
        if op in ["=", "X"]:  # Exact match or mismatch
            if op == "=":
                matches += length
            elif op == "X":
                mismatches += length
        elif op in ["I", "D"]:  # Insertions or deletions
            indels += length

    return matches, mismatches, indels


In [30]:
def calculate_def_value_enhanced(cigar_str, treat_M_as_match=True):
    """
    Enhanced function to calculate def value with optional handling of 'M' operations.

    Parameters:
    ----------
    cigar_str : str
        The CIGAR string representing the alignment.
    treat_M_as_match : bool, optional
        If True, treats 'M' operations as matches only.
        If False, treats 'M' as ambiguous (matches + mismatches).
        Default is True.

    Returns:
    -------
    float
        The def value. Returns float('inf') if sequence divergence is zero.

    Raises:
    ------
    ValueError
        If the CIGAR string contains invalid or unsupported operations.
    """
    cigar_pattern = re.compile(r'(\d+)([MIDNSHP=X])')
    parsed_cigar = cigar_pattern.findall(cigar_str)

    if not parsed_cigar:
        raise ValueError(f"Invalid CIGAR string: '{cigar_str}'")

    total_gap_length = 0
    total_mismatches = 0
    total_aligned_bases = 0

    supported_ops = {'M', '=', 'X', 'I', 'D'}

    for length_str, op in parsed_cigar:
        length = int(length_str)

        if op not in supported_ops:
            raise ValueError(f"Unsupported CIGAR operation '{op}' in CIGAR string.")

        if op in {'I', 'D'}:
            total_gap_length += length

        if op in {'M', '=', 'X'}:
            total_aligned_bases += length

        if op == 'X':
            total_mismatches += length

        if op == 'M' and not treat_M_as_match:
            # If 'M' is ambiguous, you might need additional information to split into matches and mismatches
            # For demonstration, let's assume a hypothetical mismatch rate (e.g., 10%)
            hypothetical_mismatch_rate = 0.1
            mismatches = int(length * hypothetical_mismatch_rate)
            total_mismatches += mismatches
            # Adjust aligned bases if necessary

    if total_aligned_bases > 0:
        sequence_divergence = total_mismatches / total_aligned_bases
    else:
        sequence_divergence = 0

    if sequence_divergence > 0:
        def_value = total_gap_length / sequence_divergence
    else:
        def_value = float('inf')

    return def_value


In [20]:
def_value_dictionary = {}

for key, value in results_dictionary.items():
    def_value_dictionary[key] = calculate_def_value(value[3])

In [22]:
test = results_dictionary[1]

In [23]:
cigar = test[3]

In [18]:
parse_cigar(cigar)

(0, 0, 82673)

In [26]:
import pandas as pd

s = pd.Series(list(cigar))

In [33]:
def_val = calculate_def_value_enhanced(cigar, treat_M_as_match=False)
print(f"Enhanced Def Value: {def_val}")


Enhanced Def Value: 909450.2417142858


In [27]:
s.value_counts()

M    6022
1    5009
I    3125
2    2913
D    2896
3    2099
4    1566
5    1277
6    1045
7     897
8     778
9     725
0     649
Name: count, dtype: int64

In [ ]:
# Calculate matches, mismatches, and indels
for aln in alignments:
    matches, mismatches, indels = parse_cigar(aln.cigar_str)
    aligned_length = matches + mismatches  # Only count aligned positions

    # Calculate per-base sequence divergence
    if aligned_length > 0:
        divergence = mismatches / aligned_length
        print(f"Gap-compressed per-base sequence divergence: {divergence:.4f}")
    else:
        print("No aligned positions to calculate divergence.")